In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import re
import xgboost

from pandas.api.types import CategoricalDtype
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm
from sklearn import preprocessing
from sklearn import utils

from statsmodels.graphics.tsaplots import plot_pacf
from xgboost import XGBRegressor
import lightgbm
import joblib
#from sklearn.externals import joblib
#import xgboost as xgb

In [4]:
all_csv = pd.read_csv('/content/drive/MyDrive/cleaned.csv',
                 dtype = {
                     'store_nbr' : 'category',
                     'family' : 'category',
                     'sales': 'float',
                     'city': 'category',
                     'state': 'category',
                     'type': 'category',
                     'holiday_type': 'category',
                     'holiday_transferred': 'category'
                 },
                  parse_dates=['date'])
all_csv['date'] = pd.to_datetime(all_csv['date']).dt.to_period('D')


In [5]:
all = all_csv.copy()  # we can start experimenting from here without reloading the csv file

In [6]:
# this is for experimentation

filter_by_stores = None  # note: please use string here (unlike Mine.ipynb)
filter_by_family = None
filter_by_dates = None

#filter_by_stores = ['1', '2']  # note: please use string here (unlike Mine.ipynb)
#filter_by_family = ['DAIRY', 'PRODUCE']
#filter_by_family = ['']
#filter_by_dates = '2014-06-05'

In [7]:
if filter_by_dates == None:
    train_start_date = '2013-01-01'
else:
    train_start_date = filter_by_dates
train_end_date = '2017-08-15'
test_start_date = '2017-08-16'
test_end_date = '2017-08-31'

In [8]:
if filter_by_family != None:
    all = all[all['family'].isin(filter_by_family)]
if filter_by_stores != None:
    all = all[all['store_nbr'].isin(filter_by_stores)]
if filter_by_dates != None:
    all = all[all['date'] >= filter_by_dates]

In [9]:
all.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3036528 entries, 0 to 3036527
Data columns (total 38 columns):
 #   Column               Dtype    
---  ------               -----    
 0   date                 period[D]
 1   store_nbr            category 
 2   family               category 
 3   sales                float64  
 4   onpromotion          int64    
 5   sales_lag_01         float64  
 6   sales_lag_02         float64  
 7   sales_lag_03         float64  
 8   sales_lag_04         float64  
 9   sales_lag_05         float64  
 10  sales_lag_06         float64  
 11  sales_lag_07         float64  
 12  sales_lag_08         float64  
 13  sales_lag_09         float64  
 14  sales_lag_10         float64  
 15  sales_lag_11         float64  
 16  sales_lag_12         float64  
 17  sales_lag_13         float64  
 18  sales_lag_14         float64  
 19  sales_lag_15         float64  
 20  sales_lag_16         float64  
 21  sales_lag_17         float64  
 22  sales_lag_18      

In [10]:
def one_hot_encode(df):
    return pd.get_dummies(data=df, columns=['store_nbr', 'family', 'city', 'state', 'type',
                                     'cluster', 'holiday_type', 'holiday_transferred', 'weekday']) 

In [11]:
all_ohe = one_hot_encode(all)
all_ohe = all_ohe.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))  # remove bad char in column names

X = all_ohe[all_ohe['date'] <= train_end_date]
X = X.drop(['sales'], axis=1)
y = all_ohe[['date', 'sales']][all_ohe['date'] <= train_end_date]
y.set_index('date', inplace=True)

X_test = all_ohe[all_ohe['date'] >= test_start_date]
X_test = X_test.drop(['sales'], axis=1)

X.drop('date', axis=1, inplace=True)
X_test.drop('date', axis=1, inplace=True)
y.set_index(X.index, inplace=True)

X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=1)

print(X_test)

         onpromotion  sales_lag_01  sales_lag_02  sales_lag_03  sales_lag_04  \
3008016            0      1.609438      0.693147      0.693147      1.945910   
3008017            0      0.000000      0.000000      0.000000      0.000000   
3008018            2      1.609438      1.945910      0.693147      1.386294   
3008019           20      7.571988      7.697121      6.689599      7.414573   
3008020            0      0.000000      0.000000      0.000000      0.000000   
...              ...           ...           ...           ...           ...   
3036523            1           NaN           NaN           NaN           NaN   
3036524            0           NaN           NaN           NaN           NaN   
3036525            1           NaN           NaN           NaN           NaN   
3036526            9           NaN           NaN           NaN           NaN   
3036527            0           NaN           NaN           NaN           NaN   

         sales_lag_05  sales_lag_06  sa

In [12]:

# from sklearn.decomposition import PCA
# pca = PCA(n_components=180)
# principalComponents = pca.fit_transform(X)
# principalDf = pd.DataFrame(data = principalComponents)

In [13]:
# X_train, X_val, y_train, y_val = train_test_split(principalDf, y, random_state=1)

In [14]:
# X_train.columns.size

XGBoost

In [15]:
# param = {
#       #"max_depth": [3,6,10],
#       "n_estimators": [100,110],
#       "learning_rate": [0.4,0.5],
#       "early_stopping_rounds": [60,65],
#       #'colsample_bylevel': [0.1,0.5,1]
      
#   }

In [16]:
# xgb = xgboost.XGBRegressor()

In [17]:
# grid_search = GridSearchCV(
#     estimator=xgb,
#     param_grid=param,
#     scoring = 'neg_mean_squared_error',
#     n_jobs = -1,
#     cv = 3,
#     verbose=True
# )

In [18]:
# grid_search.fit(X_train, y_train,
#         eval_set=[(X_train, y_train), (X_val, y_val)],
#         verbose=True)

In [19]:
# y_pred_train = xgb.predict(X_train)
# y_pred_train[y_pred_train < 0] = 0
# y_pred_val = xgb.predict(X_val)
# y_pred_val[y_pred_val < 0] = 0

# print("RMS log-error train: ", np.sqrt(mean_squared_log_error(y_train, y_pred_train)))
# print("RMS log-error val: ", np.sqrt(mean_squared_log_error(y_val, y_pred_val)))
# print("RMS log-error train (actual): ",
#           np.sqrt(mean_squared_log_error(np.expm1(y_train), np.expm1(y_pred_train))))
# print("RMS log-error val (actual): ",
#           np.sqrt(mean_squared_log_error(np.expm1(y_val), np.expm1(y_pred_val))))


In [20]:
!pip install -q catboost

     |████████████████████████████████| 76.6 MB 1.2 MB/s 


In [21]:
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV

In [22]:
cat = CatBoostRegressor()

In [23]:
param = {
      "max_depth": [6,8],
      "n_estimators": [1000,1400,1800],
#       "learning_rate": [0.01,0.1],
      #'colsample_bylevel': [0.1,0.5,1]
      
  }

grid_search = GridSearchCV(
    estimator=cat,
    param_grid=param,
    scoring = 'neg_mean_squared_error',
    n_jobs = -1,
    cv = 3,
    verbose=True
)

grid_search.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_val, y_val)],
        verbose=True,early_stopping_rounds=80)

grid_search.best_params_




Fitting 3 folds for each of 6 candidates, totalling 18 fits


/usr/local/lib/python3.8/dist-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Learning rate set to 0.119579
0:	learn: 2.1699279	test: 2.1699279	test1: 2.1722261	best: 2.1722261 (0)	total: 1.05s	remaining: 31m 27s
1:	learn: 1.9290333	test: 1.9290333	test1: 1.9312721	best: 1.9312721 (1)	total: 1.81s	remaining: 27m 7s
2:	learn: 1.7177334	test: 1.7177334	test1: 1.7198963	best: 1.7198963 (2)	total: 2.56s	remaining: 25m 33s
3:	learn: 1.5324429	test: 1.5324429	test1: 1.5345425	best: 1.5345425 (3)	total: 3.32s	remaining: 24m 49s
4:	learn: 1.3698968	test: 1.3698968	test1: 1.3719053	best: 1.3719053 (4)	total: 4.07s	remaining: 24m 22s
5:	learn: 1.2279041	test: 1.2279041	test1: 1.2298663	best: 1.2298663 (5)	total: 4.79s	remaining: 23m 53s
6:	learn: 1.1038905	test: 1.1038905	test1: 1.1057735	best: 1.1057735 (6)	total: 5.58s	remaining: 23m 48s
7:	learn: 0.9970014	test: 0.9970014	test1: 0.9987940	best: 0.9987940 (7)	total: 6.36s	remaining: 23m 43s
8:	learn: 0.9039737	test: 0.9039737	test1: 0.9057108	best: 0.9057108 (8)	total: 7.1s	remaining: 23m 32s
9:	learn: 0.8244460	test: 0

{'max_depth': 8, 'n_estimators': 1800}

In [33]:
import pickle
pickle.dump(grid_search, open('model_cat_gs_final.pkl', 'wb'))

In [34]:
pickled_model = pickle.load(open('/content/model_cat_gs_final.pkl', 'rb'))
pickled_model.predict(X_test)

array([1.75598268, 0.08139798, 1.76460168, ..., 0.38642649, 2.03972824,
       0.15446642])

In [35]:
y_pred_train = grid_search.predict(X_train)
y_pred_train[y_pred_train < 0] = 0
y_pred_val = grid_search.predict(X_val)
y_pred_val[y_pred_val < 0] = 0

print("RMS log-error train: ", np.sqrt(mean_squared_log_error(y_train, y_pred_train)))
print("RMS log-error val: ", np.sqrt(mean_squared_log_error(y_val, y_pred_val)))
print("RMS log-error train (actual): ",
          np.sqrt(mean_squared_log_error(np.expm1(y_train), np.expm1(y_pred_train))))
print("RMS log-error val (actual): ",
          np.sqrt(mean_squared_log_error(np.expm1(y_val), np.expm1(y_pred_val))))


RMS log-error train:  0.1364437391023863
RMS log-error val:  0.141150063024536
RMS log-error train (actual):  0.33814832447481663
RMS log-error val (actual):  0.35671279692695174


In [37]:
def main_predict(model, X_test):
    X_test_mod = X_test.copy()
    output = np.array([])
    start_day, end_day = X_test['day_of_month'].min(), X_test['day_of_month'].max()
        # we lost the dates, but we still have day_of_month, which is good enough for our experiment
        
    for day in range(start_day, end_day + 1):
        pred = model.predict(X_test_mod[X_test_mod['day_of_month'] == day])
        pred[pred < 0] = 0
        print(pred)
        output = np.concatenate([output, pred], axis=0)
        for future in range(day + 1, end_day + 1):
            X_test_mod.loc[X_test_mod[X_test_mod['day_of_month'] == future].index,
                           f'sales_lag_{(future - day):02d}'] = pred
            # fill out future values now that this sales figure is available
            
    return output

In [38]:
y_pred_test = main_predict(pickled_model, X_test)

[1.75598268 0.08139798 1.76460168 ... 7.40719359 4.57347314 2.76531301]
[1.63430894 0.03918672 1.57736059 ... 7.09827602 4.55466033 2.71242907]
[1.67380261 0.02336093 1.50447575 ... 7.10483246 4.39049955 2.64939885]
[1.71105681 0.         1.53054916 ... 7.3259278  4.78024072 3.01473138]
[1.34752496 0.         1.16406382 ... 7.51098917 5.14435481 3.06123252]
[1.73567912 0.07245655 1.72386337 ... 7.23275168 4.81822313 2.78782287]
[1.64341669 0.04506791 1.51723442 ... 7.75337021 4.58544186 2.737514  ]
[1.68430443 0.04985845 1.62012876 ... 7.16448198 4.3229636  2.64247399]
[1.63454485 0.         1.51071403 ... 7.06071032 4.49042416 2.62854002]
[1.65297415 0.0096225  1.40972626 ... 7.05485068 4.22907829 2.58599987]
[1.76321342 0.         1.61862773 ... 7.29628881 4.7453972  3.12782083]
[1.24447824 0.         1.17547968 ... 7.48775175 5.10468301 3.06868142]
[1.65717151 0.05821054 1.72815352 ... 7.16986575 4.68995149 2.65766637]
[1.65874565 0.04823647 1.71804979 ... 7.71736614 4.52665648 2.68

In [42]:
delta_index = 3008016 - 3000888  # we inserted 4 Christmas days, 4 x 54 x 33 = 7128, which is the difference
submission = pd.DataFrame({'id': X_test.index - delta_index, 'sales': np.expm1(y_pred_test)})
submission.to_csv('submissioncatgsfinal.csv', index=False)